### 1. Import library

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")

# 모듈 설치
!pip install konlpy
!pip install regex
!pip --version
!pip install --upgrade pip
!pip install wget
!pip install whoosh
!pip install transformers
!pip install sentence_transformers
!pip install fairseq
!pip install kss

# 데이터 처리 모듈
import pandas as pd
import copy
import re
import math
import json
import requests
import urllib.request
from tqdm import tqdm
import datetime
import time
import os

# 텍스트 관련 모듈
from konlpy.tag import Okt
okt = Okt()
from konlpy.tag import *
import nltk

# 전처리 모듈
from sklearn.preprocessing import MinMaxScaler

#시각화 모듈
import matplotlib.pyplot as plt
import seaborn as sns

Mounted at /content/drive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 51.0 MB/s eta 0:00:00
pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 16.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9657 sha256=3cd23b32f8d40f9d236e43c966e58e96edfb010bc6085e40e3145e1bc76f2b49
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.8/468.8 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... d

### 2. Summarize




#### pororo 설치 방법
- https://dacon.io/competitions/official/235875/codeshare/4462



In [ ]:
# 모듈 설치(한번만 하고 주석)
!git clone https://github.com/kakaobrain/pororo.git

# 경로 설정(한번만 하고 주석)
%cd pororo

# 패키지 설치(한번만 하고 주석)
!pip install -e .

Cloning into 'pororo'...
remote: Enumerating objects: 2119, done.
remote: Counting objects: 100% (29/29), done.
remote: Compressing objects: 100% (21/21), done.
remote: Total 2119 (delta 11), reused 22 (delta 8), pack-reused 2090
Receiving objects: 100% (2119/2119), 13.09 MiB | 7.63 MiB/s, done.
Resolving deltas: 100% (1448/1448), done.
/content/pororo
Obtaining file:///content/pororo
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of pororo to determine which version is compatible with other requirements. This could take a while.
ERROR: Could not find a version that satisfies the requirement torch==1.6.0 (from pororo) (from versions: 1.11.0, 1.12.0, 1.12.1, 1.13.0, 1.13.1, 2.0.0, 2.0.1, 2.1.0, 2.1.1)
ERROR: No matching distribution found for torch==1.6.0


In [ ]:
# 모듈 import
from pororo import Pororo
# pororo 기반 문서 요약 모델
abs_summ = Pororo(task="text_summarization", lang="ko", model="abstractive")

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


tokenizer_config.json:   0%|          | 0.00/337 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.05M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/109 [00:00<?, ?B/s]

In [ ]:
# 경로 수정할 때 사용(이전 경로 삭제됨)
%cd ..

/content


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/hana/data/target_summarize/sample.csv', encoding='cp949')

In [ ]:
df

,user,indexing,content,target
0,user1,"삼성전자 0.3, LG 0.2, 현대자동차 0.5",삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자...,"코스피 추후 주가 방향성과 삼성전자, 엘지, 현대자동차 등의 구체적인 구성 요소에 ..."
1,user2,"삼성전자 0.4, LG화학 0.3, SK이노베이션 0.3",삼성전자는 글로벌 반도체 시장에서의 지배적인 입지와 안정적인 실적으로 향후 안정적인...,"변동성이 적은 코스피 주가를 활용한 다이렉트 인덱싱 구성, 1년 이상 장기 투자자 대상"
2,user3,"삼성바이오로직스 0.3, SK텔레콤 0.2, 네이버 0.5",삼성바이오로직스는 생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로 인해...,일별 뉴스 데이터 정리를 통한 등락 예측으로 최근 꾸준한 성장세와 ROI가 높은 주...


In [ ]:
# Summarizing news data : 코스피 데이터 요약
summarized_li = []
for content in tqdm(df['target']):
    try:
        if len(content) > 0:
            # abs_summ 함수를 사용하여 뉴스 내용을 요약
            summarized_contents = abs_summ(content,
                                            beam=5,
                                            len_penalty=0.6,
                                            no_repeat_ngram_size=3,
                                            top_k=50,
                                            top_p=0.7)
            summarized_li.append(summarized_contents)
        else:
            summarized_li.append("")  # Handle empty content
    except IndexError:
        print("IndexError: Skipping this article")
        summarized_li.append("")  # Handle the error by adding an empty string

# Add summarized contents to the DataFrame
df['summarize_target'] = summarized_li

100%|██████████| 3/3 [00:01<00:00,  1.63it/s]


In [ ]:
df.to_csv('/content/drive/MyDrive/hana/data/target_summarize/sample_plus_target_summarize.csv', encoding='cp949', index=False)

In [ ]:
# 포트폴리오 비율 삼성전자 0.4, lg 0.2

# 추천 대상 / 장기 투자자 전기 테마 위주로 찾는 사람에게 추천한다

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
input_str = "미래 장기 투자를 위한 포트폴리오"

In [ ]:
# CountVectorizer를 사용하여 벡터화하고 코사인 유사도 계산
def calculate_cosine_similarity(str1, str2):
    vectorizer = CountVectorizer().fit_transform([str1, str2])
    vectors = vectorizer.toarray()
    cosine_sim = cosine_similarity(vectors)
    return cosine_sim[0][1]

# 각 행에 대해 코사인 유사도 계산하여 새로운 열 'cosine_similarity' 추가
df['cosine_similarity'] = df.apply(lambda row: calculate_cosine_similarity(input_str, row['summarize_target']), axis=1)


In [ ]:
df

,user,indexing,content,target,summarize_target,cosine_similarity
0,user1,"삼성전자 0.3, LG 0.2, 현대자동차 0.5",삼성전자는 글로벌 반도체 시장에서의 선도적인 입지와 안정적인 실적으로 안정적인 투자...,"코스피 추후 주가 방향성과 삼성전자, 엘지, 현대자동차 등의 구체적인 구성 요소에 ...","삼성전자 구성 요소에 대한 소개 소개 삼성전자, 엘지, 현대자동차 등의 구체적인...",0.000000
1,user2,"삼성전자 0.4, LG화학 0.3, SK이노베이션 0.3",삼성전자는 글로벌 반도체 시장에서의 지배적인 입지와 안정적인 실적으로 향후 안정적인...,"변동성이 적은 코스피 주가를 활용한 다이렉트 인덱싱 구성, 1년 이상 장기 투자자 대상","변동렉트 인덱싱 구성, 1년 이상 장기 투자자 대상 대상 코스피 변동 변동성...",0.083045
2,user3,"삼성바이오로직스 0.3, SK텔레콤 0.2, 네이버 0.5",삼성바이오로직스는 생명과학 분야에서의 선도적인 위치와 신약 개발에 대한 투자로 인해...,일별 뉴스 데이터 정리를 통한 등락 예측으로 최근 꾸준한 성장세와 ROI가 높은 주...,뉴스별 뉴스 데이터 정리를 통한 등락 예측으로 최근 꾸준한 성장세와 ROI가 높은 ...,0.000000
